In [ ]:
import pandas as pd
import numpy as np
from catboost import CatBoostClassifier
import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score, average_precision_score
from sklearn.preprocessing import OrdinalEncoder
import warnings
warnings.filterwarnings('ignore')

print("1. Đang đọc dữ liệu...")
# Tuyệt đối không xài absolute paths, chỉ dùng file name trong thư mục hiện tại
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

target_col = 'underperforming'
id_col = 'id'

# ---------------------------------------------------------
# 2. FEATURE ENGINEERING (Kỹ nghệ đặc trưng)
# ---------------------------------------------------------
print("2. Đang tạo thêm đặc trưng (Feature Engineering)...")

def feature_engineering(df):
    df_new = df.copy()
    
    # Đặc trưng tỷ lệ
    df_new['capacity_per_age'] = df_new['capacity_mw'] / (df_new['plant_age'] + 1)
    df_new['log_cap_per_age'] = df_new['capacity_log_mw'] / (df_new['plant_age'] + 1)
    
    # Đặc trưng không gian (Xoay trục tọa độ 45 độ giúp Decision Tree chia cắt tốt hơn)
    df_new['rot_45_x'] = 0.707 * df_new['latitude'] + 0.707 * df_new['longitude']
    df_new['rot_45_y'] = 0.707 * df_new['longitude'] - 0.707 * df_new['latitude']
    df_new['distance_from_center'] = np.sqrt(df_new['latitude']**2 + df_new['longitude']**2)
    
    # Đặc trưng tương tác Categorical
    df_new['fuel_x_capacity'] = df_new['primary_fuel'].astype(str) + "_" + df_new['capacity_band'].astype(str)
    
    return df_new

train_fe = feature_engineering(train)
test_fe = feature_engineering(test)

# Các cột phân loại ban đầu + cột mới tạo
cat_features = [
    'fuel_group', 'primary_fuel', 'other_fuel1', 
    'owner_bucket', 'capacity_band', 'lat_band', 'lon_band',
    'fuel_x_capacity'
]

features = [col for col in train_fe.columns if col not in [id_col, target_col]]

# Xử lý biến phân loại cho CatBoost (dạng chuỗi)
for col in cat_features:
    train_fe[col] = train_fe[col].astype(str)
    test_fe[col] = test_fe[col].astype(str)

# Xử lý biến phân loại cho LightGBM (dạng Ordinal Categorical)
train_lgb = train_fe.copy()
test_lgb = test_fe.copy()

oe = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)
train_lgb[cat_features] = oe.fit_transform(train_lgb[cat_features])
test_lgb[cat_features] = oe.transform(test_lgb[cat_features])

for col in cat_features:
    train_lgb[col] = train_lgb[col].astype('category')
    test_lgb[col] = test_lgb[col].astype('category')

# ---------------------------------------------------------
# 3. K-FOLD CROSS VALIDATION & ENSEMBLING
# ---------------------------------------------------------
n_splits = 10 # Nâng lên 10 folds để mô hình học sâu hơn
skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

cb_test_preds = np.zeros(len(test))
lgb_test_preds = np.zeros(len(test))
cv_scores = []

print(f"3. Bắt đầu huấn luyện Ensemble (CatBoost + LightGBM) với {n_splits}-Fold...\n")

for fold, (train_idx, val_idx) in enumerate(skf.split(train_fe[features], train_fe[target_col])):
    y_train = train_fe.loc[train_idx, target_col]
    y_val = train_fe.loc[val_idx, target_col]
    
    # --- MÔ HÌNH 1: CATBOOST ---
    X_train_cb, X_val_cb = train_fe.loc[train_idx, features], train_fe.loc[val_idx, features]
    
    cb_model = CatBoostClassifier(
        iterations=2000, learning_rate=0.02, depth=6,
        cat_features=cat_features, eval_metric='Logloss',
        random_seed=42 + fold, verbose=0, task_type='CPU'
    )
    cb_model.fit(X_train_cb, y_train, eval_set=(X_val_cb, y_val), early_stopping_rounds=200, verbose=False)
    cb_val_preds = cb_model.predict_proba(X_val_cb)[:, 1]
    
    # --- MÔ HÌNH 2: LIGHTGBM ---
    X_train_lgb, X_val_lgb = train_lgb.loc[train_idx, features], train_lgb.loc[val_idx, features]
    
    lgb_model = lgb.LGBMClassifier(
        n_estimators=2000, learning_rate=0.02, max_depth=7,
        num_leaves=31, subsample=0.8, colsample_bytree=0.8,
        random_state=42 + fold, verbose=-1, n_jobs=-1
    )
    # LightGBM hỗ trợ early stopping thông qua callbacks
    callbacks = [lgb.early_stopping(stopping_rounds=200, verbose=False)]
    lgb_model.fit(
        X_train_lgb, y_train, 
        eval_set=[(X_val_lgb, y_val)],
        callbacks=callbacks
    )
    lgb_val_preds = lgb_model.predict_proba(X_val_lgb)[:, 1]
    
    # --- BLEND (Kết hợp 50/50) ---
    val_preds = (cb_val_preds + lgb_val_preds) / 2
    
    roc_auc = roc_auc_score(y_val, val_preds)
    ap = average_precision_score(y_val, val_preds)
    fold_score = 0.7 * roc_auc + 0.3 * ap
    cv_scores.append(fold_score)
    
    print(f"Fold {fold+1:02d} | ROC AUC: {roc_auc:.4f} | AP: {ap:.4f} | Score: {fold_score:.4f}")
    
    # Lưu dự đoán tập test
    cb_test_preds += cb_model.predict_proba(test_fe[features])[:, 1] / n_splits
    lgb_test_preds += lgb_model.predict_proba(test_lgb[features])[:, 1] / n_splits

# ---------------------------------------------------------
# 4. LƯU KẾT QUẢ
# ---------------------------------------------------------
print(f"\n---> ĐIỂM SỐ TRUNG BÌNH (Local CV): {np.mean(cv_scores):.5f}")

# Kết hợp dự đoán test từ cả 2 mô hình
final_test_preds = (cb_test_preds + lgb_test_preds) / 2

submission = pd.DataFrame({
    'id': test['id'],
    'underperforming': final_test_preds
})

file_name = 'submission_out_trinh.csv'
submission.to_csv(file_name, index=False)
print(f"Đã tạo thành công '{file_name}'! Nộp bài thôi!")